In [1]:
from google.cloud import bigquery
import pandas as pd
from google.oauth2 import service_account

In [2]:
credentials = service_account.Credentials.from_service_account_file('C:\\Users\\elija\\Documents\\24f-coop\\credentials.json')
project = 'net-data-viz-handbook'
# Initialize a GCS client
client = bigquery.Client(credentials=credentials, project=project)

In [35]:
def build_geographic_filter(geo_level, geo_values, alias="g_target"):
    # Accepts a geographic level (e.g., 'country_id', 'region_label') and a list of values
    if isinstance(geo_values, list):
        # Check the type of the first item to determine the filter type
        if isinstance(geo_values[0], int):
            values = ', '.join(str(val) for val in geo_values)  # For INT64
            return f"{alias}.{geo_level} IN ({values})"
        elif isinstance(geo_values[0], str):
            values = ', '.join(f"'{val}'" for val in geo_values)  # For STRING
            return f"{alias}.{geo_level} IN ({values})"
    else:
        # Handle single values
        if isinstance(geo_values, int):
            return f"{alias}.{geo_level} = {geo_values}"
        elif isinstance(geo_values, str):
            return f"{alias}.{geo_level} = '{geo_values}'"

In [47]:
def build_query(table_name, source_geo_level, target_geo_level, source_values, target_values, domestic=True, cutoff=0.05, output_geo_level=None):
    # Build filters for both source and target regions
    source_filter = build_geographic_filter(source_geo_level, source_values, alias="g_source") if source_values is not None else 'TRUE'
    target_filter = build_geographic_filter(target_geo_level, target_values, alias="g_target") if target_values is not None else 'TRUE'
    
    # Create the base where clause
    where_clauses = [target_filter, source_filter]
    
    if not domestic:
        # Exclude rows where target imports to itself
        where_clauses.append(f"g_source.{target_geo_level} <> g_target.{target_geo_level}")

    # Join the where clauses with 'AND'
    where_clause = ' AND '.join(where_clauses)

    query = f"""
    WITH region_imports AS (
      SELECT 
        g_source.{output_geo_level} AS source_label, 
        SUM(i.importations) AS total_importations
      FROM 
        `{table_name}` AS i
      JOIN 
        `net-data-viz-handbook.reference.gleam-geo-map` AS g_source 
        ON g_source.basin_id = i.source_basin
      JOIN 
        `net-data-viz-handbook.reference.gleam-geo-map` AS g_target 
        ON g_target.basin_id = i.target_basin
      WHERE 
        {where_clause}  
      GROUP BY 
        g_source.{output_geo_level}
    ),
    total_imports AS (
      SELECT 
        SUM(total_importations) AS grand_total_importations 
      FROM region_imports
    ),
    categorized_regions AS (
      SELECT 
        r.source_label,
        CASE 
          WHEN r.total_importations < ({cutoff} * (SELECT grand_total_importations FROM total_imports)) THEN 'Other'
          ELSE r.source_label
        END AS categorized_label
      FROM 
        region_imports r
    )
    SELECT 
      cr.categorized_label AS source_label, 
      i.date, 
      SUM(i.importations) AS importations,
      AVG(SUM(i.importations)) OVER (
        PARTITION BY cr.categorized_label 
        ORDER BY i.date 
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
      ) AS rolling_importations
    FROM 
      `{table_name}` AS i
    JOIN 
      `net-data-viz-handbook.reference.gleam-geo-map` AS g_target 
      ON g_target.basin_id = i.target_basin
    JOIN 
      `net-data-viz-handbook.reference.gleam-geo-map` AS g_source 
      ON g_source.basin_id = i.source_basin
    JOIN 
      categorized_regions cr 
      ON cr.source_label = g_source.{output_geo_level}
    WHERE 
      {where_clause}
    GROUP BY 
      cr.categorized_label, 
      i.date
    ORDER BY 
      i.date
    """

    return query

In [39]:
def execute_query_and_return_df(client, query):
    # Execute query
    query_job = client.query(query)

    # Convert result to a Pandas DataFrame
    result_df = query_job.to_dataframe()

    return result_df

In [51]:
table_name = 'net-data-viz-handbook.importation_data.sir_importations_1'
source_geo_level = 'continent_label'  # Geographic level for source filtering
target_geo_level = 'country_id'        # Geographic level for target filtering
output_geo_level = 'region_label'      # Geographic level for output

source_values = None
target_values = [218]                   # Filter targets by country
domestic = False

# Build the SQL query
query = build_query(table_name, source_geo_level, target_geo_level, source_values, target_values, 
                    domestic=domestic, cutoff=0.05, output_geo_level=output_geo_level)

# Execute the query and return the DataFrame
df = execute_query_and_return_df(client, query)

df.tail()

C:\Users\elija\anaconda3\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,source_label,date,importations,rolling_importations
1491,Caribbean,2010-02-17,33,49.142857
1492,Northern Europe,2010-02-17,15,13.000000
1493,Northern America,2010-02-17,9,6.714286
1494,Eastern Asia,2010-02-17,86,96.142857
1495,Other,2010-02-17,157,172.571429
